In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

import dask.dataframe as dd

import pyarrow.dataset as ds
import rpy2_arrow.pyarrow_rarrow as pyra
import pyarrow.feather as feather
import pyarrow.parquet as pq

In [3]:
%load_ext rpy2.ipython
%load_ext memory_profiler

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# DSCI 525 - Web and Cloud Computing

## Milestone 1: Tackling big data on your laptop

### 1. Downloading the data

Download and extract the data.zip folder from figshare

In [4]:
%%time
%%memit

article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "data/"

response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want

files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

peak memory: 278.85 MiB, increment: 3.40 MiB
Wall time: 7min 59s


In [5]:
%%time
%%memit      
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)
    
os.remove("data/data.zip")
os.remove("data/observed_daily_rainfall_SYD.csv")

peak memory: 283.99 MiB, increment: 6.42 MiB
Wall time: 25.7 s


### 2. Combining data CSVs

- Merge the files using pandas
- Run times for combining CSV files

  - *Jingjing*
         - peak memory: 431.68 MiB, increment: 0.25 MiB
         - CPU times: user 5min 40s, sys: 23.9 s, total: 5min 46s
         - Wall time: 6min 27s  

In [6]:
use_cols = ['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)']
files = glob.glob('data/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
                for file in files)
              )

Save the combined dataframe

In [14]:
df.to_csv('combined.csv')

### 4. Perform a simple EDA in R

##### Pandas format

In [4]:
%%time
%%memit
df = pd.read_csv("combined.csv", nrows=1_000_000)

peak memory: 412.40 MiB, increment: 136.86 MiB
Wall time: 1.7 s


##### Arrow format

In [8]:
%%time
%%memit
dataset = ds.dataset("combined.csv", format="csv")
table = dataset.to_table()

peak memory: 4552.66 MiB, increment: 4181.79 MiB
Wall time: 36.5 s


##### Feather format

In [9]:
%%time
%%memit
feather.write_feather(table, 'combined.feather')

peak memory: 4566.12 MiB, increment: 13.46 MiB
Wall time: 4.93 s


##### Parquet format

In [10]:
%%time
%%memit
pq.write_table(table, 'combined.parquet')

peak memory: 4708.31 MiB, increment: 142.86 MiB
Wall time: 16.5 s


In [11]:
%%sh
# csv data
du -sh combined.csv

# feather data
du -sh combined.feather

# parquet data
du -sh combined.parquet

6.0G	combined.csv
1.1G	combined.feather
542M	combined.parquet


#### Transfer the dataframe from python to R and perform EDA

In [13]:
%%R
library(dplyr)
library(arrow)

##### Pandas

In [14]:
%%time
%%R -i df

start_time <- Sys.time()
print(class(df))
result <- df %>% collect() %>% summary()
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "data.frame"
     time              lat_min          lat_max          lon_min     
 Length:1000000     Min.   :-36.25   Min.   :-35.00   Min.   :140.6  
 Class :character   1st Qu.:-36.25   1st Qu.:-35.00   1st Qu.:142.5  
 Mode  :character   Median :-35.00   Median :-33.75   Median :146.2  
                    Mean   :-34.92   Mean   :-33.67   Mean   :146.1  
                    3rd Qu.:-33.75   3rd Qu.:-32.50   3rd Qu.:150.0  
                    Max.   :-32.50   Max.   :-31.25   Max.   :151.9  
    lon_max      rain (mm/day)         model          
 Min.   :142.5   Min.   :  0.0000   Length:1000000    
 1st Qu.:144.4   1st Qu.:  0.0082   Class :character  
 Median :148.1   Median :  0.1190   Mode  :character  
 Mean   :147.9   Mean   :  1.7961                     
 3rd Qu.:151.9   3rd Qu.:  0.8148                     
 Max.   :153.8   Max.   :432.9395                     
Time difference of 0.2850361 secs
Wall time: 46.2 s


##### Arrow

In [15]:
r_table = pyra.converter.py2rpy(table)

6052
rarrow.ChunkedArray: 0.0380098819732666
6052
rarrow.ChunkedArray: 0.028034687042236328
6052
rarrow.ChunkedArray: 0.03199338912963867
6052
rarrow.ChunkedArray: 0.048981428146362305
6052
rarrow.ChunkedArray: 0.0460052490234375
6052
rarrow.ChunkedArray: 0.029996156692504883
6052
rarrow.ChunkedArray: 0.030958175659179688


In [16]:
%%time
%%R -i r_table

start_time <- Sys.time()
print(class(r_table))
result <- r_table %>% collect() %>% summary()
print(class(r_table %>% collect()))
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "Table"       "ArrowObject" "R6"         
[1] "tbl_df"     "tbl"        "data.frame"
      time                        lat_min           lat_max      
 Min.   :1889-01-01 08:06:00   Min.   :-36       Min.   :-36.00  
 1st Qu.:1920-07-02 20:00:00   1st Qu.:-35       1st Qu.:-33.66  
 Median :1952-01-01 20:00:00   Median :-33       Median :-32.04  
 Mean   :1952-01-02 00:32:09   Mean   :-33       Mean   :-31.98  
 3rd Qu.:1983-07-02 20:00:00   3rd Qu.:-31       3rd Qu.:-30.16  
 Max.   :2014-12-31 20:00:00   Max.   :-30       Max.   :-27.91  
                               NA's   :3219300                   
    lon_min           lon_max      rain (mm/day)        model          
 Min.   :141       Min.   :141.2   Min.   :  0       Length:62467843   
 1st Qu.:143       1st Qu.:145.0   1st Qu.:  0       Class :character  
 Median :147       Median :148.1   Median :  0       Mode  :character  
 Mean   :147       Mean   :148.2   Mean   :  2                         
 3rd Qu.:150       3rd 

##### Feather format

In [ ]:
# %%time
# %%R
# start_time <- Sys.time()
# r_table <- arrow::read_feather("combined.feather")
# print(class(r_table))
# result <- r_table %>% summary()
# end_time <- Sys.time()
# print(result)
# print(end_time - start_time)

##### Parquet format

In [5]:
%%time
%%R

start_time <- Sys.time()
r_table <- arrow::read_parquet("combined.parquet")
print(class(r_table))
library(dplyr)
result <- r_table %>% summary()
end_time <- Sys.time()
print(result)
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"


R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




      time                        lat_min           lat_max      
 Min.   :1889-01-01 08:06:00   Min.   :-36       Min.   :-36.00  
 1st Qu.:1920-07-02 20:00:00   1st Qu.:-35       1st Qu.:-33.66  
 Median :1952-01-01 20:00:00   Median :-33       Median :-32.04  
 Mean   :1952-01-02 00:32:09   Mean   :-33       Mean   :-31.98  
 3rd Qu.:1983-07-02 20:00:00   3rd Qu.:-31       3rd Qu.:-30.16  
 Max.   :2014-12-31 20:00:00   Max.   :-30       Max.   :-27.91  
                               NA's   :3219300                   
    lon_min           lon_max      rain (mm/day)        model          
 Min.   :141       Min.   :141.2   Min.   :  0       Length:62467843   
 1st Qu.:143       1st Qu.:145.0   1st Qu.:  0       Class :character  
 Median :147       Median :148.1   Median :  0       Mode  :character  
 Mean   :147       Mean   :148.2   Mean   :  2                         
 3rd Qu.:150       3rd Qu.:151.3   3rd Qu.:  1                         
 Max.   :154       Max.   :155.6   Max. 

#### Discussion

From above results, our group would choose parquet format to transfer the dataframe from python to R. First, the file size of our combined data is 542M with parquet format, which is much smaller than the file sizes with csv (6.0G) and feather (1.1G) format. Second, the wall time for parquet format is shorter than the wall time for feather format. The wall time for csv is a little bit shorter than the wall time for parquet. However, consider both the file size and speed, it would be better and reasonable to choose parquet format to transfer the dataframe from python to R.

##### More simpe EDA with Parquet format

In [6]:
%%time
%%R
r_table <- arrow::read_parquet("combined.parquet")
class(r_table)

[1] "tbl_df"     "tbl"        "data.frame"
Wall time: 9.31 s


In [7]:
%%time
%%R
head(r_table)

# A tibble: 6 x 7
  time                lat_min lat_max lon_min lon_max `rain (mm/day)` model     
  <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>     
1 1889-01-01 20:06:00   -36.2     -35    141.    142.        3.29e-13 "data\\AC~
2 1889-01-02 20:06:00   -36.2     -35    141.    142.        0.       "data\\AC~
3 1889-01-03 20:06:00   -36.2     -35    141.    142.        0.       "data\\AC~
4 1889-01-04 20:06:00   -36.2     -35    141.    142.        0.       "data\\AC~
5 1889-01-05 20:06:00   -36.2     -35    141.    142.        1.05e- 2 "data\\AC~
6 1889-01-06 20:06:00   -36.2     -35    141.    142.        3.29e- 2 "data\\AC~
Wall time: 3.17 s


In [8]:
%%time
%%R
tail(r_table)

# A tibble: 6 x 7
  time                lat_min lat_max lon_min lon_max `rain (mm/day)` model     
  <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>     
1 2014-12-26 20:00:00   -30.2   -29.2    153.    154.           0.435 "data\\Ta~
2 2014-12-27 20:00:00   -30.2   -29.2    153.    154.           0.554 "data\\Ta~
3 2014-12-28 20:00:00   -30.2   -29.2    153.    154.           7.03  "data\\Ta~
4 2014-12-29 20:00:00   -30.2   -29.2    153.    154.           0.235 "data\\Ta~
5 2014-12-30 20:00:00   -30.2   -29.2    153.    154.           2.10  "data\\Ta~
6 2014-12-31 20:00:00   -30.2   -29.2    153.    154.           0.548 "data\\Ta~
Wall time: 501 ms


### Reflection

- Challenges with downloading the data
    - It takes really long time to download the large size of data because of the internet connection. I fail to download the data with my own VPN. I successfully download the data by using UBC VPN
    
- Challenges with performing a simple EDA in R
    - It takes really long time to transfer dataframe from python to R with feather format. Like more than an hour. I try to restart PC and Jupyter Lab but doesn't work. I comment the code cell for feather format and get the conclusion that it takes longer time than the other formats.